# Day 13 - Transparent Origami

https://adventofcode.com/2021/day/13

In [43]:
from pathlib import Path

INPUTS = Path("input.txt").read_text().strip()

# Separate the inputs into the set of coordinates and the fold instructions
COORDINATES, folds = INPUTS.split("\n\n")

# Process the coordinates into tuples of integers representing (X, Y)
COORDINATES = [tuple(map(int, x.split(","))) for x in COORDINATES.split("\n")]

# Process the folds into tuples of (axis, num)
FOLDS = []
for fold in folds.split("\n"):
    instruction = fold[11:]
    axis, num = instruction.split("=")
    FOLDS.append((axis, int(num)))

## Part 1

For this first step, we just need to know the number of visible points after the first fold. I'm choosing to do so using sets to eliminate duplicate points, and then just count the length.

The trick will be processing each point by assessing whether it is above/left or below/right of the fold, then adjusting its "folded" coordinates to match. That way, folded coordinates will be added to the fold set and eliminated if they are duplicates of another set.

In [44]:
def fold_coords(
    inputs: list[tuple[int, int]],
    axis: str,
    num: int,
) -> list[tuple[int, int]]:
    new_coords = set()
    for coord_x, coord_y in inputs:
        new_x, new_y = coord_x, coord_y
        if axis == "y" and new_y > num:
            # Adjust the new Y to be above the fold
            new_y -= (new_y - num) * 2
        elif axis == "x" and new_x > num:
            # Adjust the new X to be left of the fold
            new_x -= (new_x - num) * 2
        new_coords.add((new_x, new_y))
    return list(new_coords)

Some testing using the examples from the site shows some plots small enough that I can guess they're accurate, without actually plotting them to see their output yet.

In [45]:
def test_fold_coords():
    coords = [
        (6, 10),
        (0, 14),
        (9, 10),
        (0, 3),
        (10, 4),
        (4, 11),
        (6, 0),
        (6, 12),
        (4, 1),
        (0, 13),
        (10, 12),
        (3, 4),
        (3, 0),
        (8, 4),
        (1, 10),
        (2, 14),
        (8, 10),
        (9, 0),
    ]
    new_coords = fold_coords(inputs=coords, axis="y", num=7)
    expected = [
        (0, 0),
        (0, 1),
        (0, 3),
        (1, 4),
        (2, 0),
        (3, 0),
        (3, 4),
        (4, 1),
        (4, 3),
        (6, 0),
        (6, 2),
        (6, 4),
        (8, 4),
        (9, 0),
        (9, 4),
        (10, 2),
        (10, 4),
    ]
    assert sorted(new_coords) == expected

    new_coords2 = fold_coords(inputs=new_coords, axis="x", num=5)
    expected2 = [
        (0, 0),
        (0, 1),
        (0, 2),
        (0, 3),
        (0, 4),
        (1, 0),
        (1, 4),
        (2, 0),
        (2, 4),
        (3, 0),
        (3, 4),
        (4, 0),
        (4, 1),
        (4, 2),
        (4, 3),
        (4, 4),
    ]
    assert sorted(new_coords2) == expected2


test_fold_coords()

Now to get our solution, the number of coordinates visible after a single fold.

In [46]:
axis, num = FOLDS[0]
new_coords = fold_coords(inputs=COORDINATES, axis=axis, num=num)
print(len(new_coords))

802


## Part 2

Performing all the folds now should be trivial, but then we have to actually print the thing and see what the output looks like.

First things first, got to get the set of coordinates after all the folds are done.

In [47]:
from copy import deepcopy

coords = deepcopy(COORDINATES)
for axis, num in FOLDS:
    coords = fold_coords(inputs=coords, axis=axis, num=num)

final_coords = coords

Then we can get our output from those coordinates.

In [48]:
def coord_plot(coords: list[tuple[int, int]]) -> str:
    max_x = max([x[0] for x in coords])
    max_y = max([x[1] for x in coords])

    output = []
    # The coordinates include the max, so add 1 to it on each loop,
    # or we'll be missing a line below and to the right (and won't be able to read it!)
    for y in range(max_y + 1):
        line = ""
        for x in range(max_x + 1):
            char = " "
            if (x, y) in coords:
                char = "#"
            line += char
        output.append(line)
    return "\n".join(output)


plot = coord_plot(final_coords)
print(plot)

###  #  # #  # #### ####  ##  #  # ### 
#  # # #  #  # #       # #  # #  # #  #
#  # ##   #### ###    #  #    #  # ### 
###  # #  #  # #     #   # ## #  # #  #
# #  # #  #  # #    #    #  # #  # #  #
#  # #  # #  # #    ####  ###  ##  ### 


Looks to me like "RKHFZGUB", and the site says that's correct. Woot!